<a href="https://colab.research.google.com/github/davdevv/ML_position_assignment/blob/main/Task_2/notebooks/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

#Клонируем репозиторий


In [2]:
!git clone https://github.com/davdevv/ML_position_assignment.git

Cloning into 'ML_position_assignment'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 165 (delta 17), reused 1 (delta 0), pack-reused 121
Receiving objects: 100% (165/165), 939.28 KiB | 5.80 MiB/s, done.
Resolving deltas: 100% (58/58), done.


#Скачиваем val датасет с диска

In [3]:
!gdown '1WPopTKn4OL6YwTRiiR-7RDsRHRfyARMF'

Downloading...
From: https://drive.google.com/uc?id=1WPopTKn4OL6YwTRiiR-7RDsRHRfyARMF
To: /content/val.rar
100% 373M/373M [00:11<00:00, 31.8MB/s]


#Скачиваем чекпоинт

In [4]:
!gdown '1AHZ8soHuE7WalZJ1U1gWuYUFrFoXWV31'

Downloading...
From: https://drive.google.com/uc?id=1AHZ8soHuE7WalZJ1U1gWuYUFrFoXWV31
To: /content/audio_denoising_chpt.pth
100% 654k/654k [00:00<00:00, 132MB/s]


In [ ]:
!pip install unrar
!unrar x val

#Валидационная функция

In [6]:
!pip install pytorch_msssim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from ML_position_assignment.Task_2.model import ResidualDenseNetwork as Model
from ML_position_assignment.Task_2.metrics import PSNR, SSIM
from ML_position_assignment.Task_2.dataset import load, CustomDataset

In [11]:
def validate(
        loader,
        model,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
):
    model.eval()
    psnr = PSNR(data_range=1, reduction='mean')
    ssim = SSIM(1, data_range=1, reduction='mean')
    with torch.no_grad():
        val_psnr = []
        val_ssim = []
        val_mse = []
        for clean_image, noisy_image in tqdm(loader):
            noisy_image = noisy_image.to(device, dtype=torch.float)
            clean_image = clean_image.to(device, dtype=torch.float)
            prediction = model(noisy_image)
            current_psnr = psnr(clean_image, prediction).item()
            current_ssim = ssim(clean_image, prediction).item()
            mse = nn.MSELoss()
            current_mse = mse(clean_image, prediction)

            val_psnr.append(current_psnr)
            val_ssim.append(current_ssim)
            val_mse.append(current_mse.cpu())

    val_psnr = np.mean(val_psnr)
    val_ssim = np.mean(val_ssim)
    val_mse = np.mean(val_mse)
    print("\nVal MSE: {:.5f}".format(val_mse))
    print("\nMean Test PSNR: {:.2f}\nMean Test SSIM: {:.2f}".format(val_psnr, val_ssim))
    print('-' * 50)

    return val_psnr, val_ssim

#Тестируем модель на валидационной выборке

In [9]:
kernel_size = 3
num_channels = 1
growth_rate = 10
num_features = 12
num_blocks = 7
num_layers = 6

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(kernel_size, num_channels, growth_rate,
              num_features, num_blocks, num_layers).to(device)
model.load_state_dict(torch.load('audio_denoising_chpt.pth',
                                 map_location=device))
val_loader = load('val/val', batch_size=1, validate=True)
validate(val_loader, model, device)

100%|██████████| 2000/2000 [00:41<00:00, 48.10it/s]


Val MSE: 0.03617

Mean Test PSNR: 15.53
Mean Test SSIM: 0.60
--------------------------------------------------


(15.525494769454003, 0.5998819938749075)